In [1]:
import pandas as pd
import numpy as np

In [2]:
df= pd.read_csv('ccdefault.csv')
df.drop(['ID'],axis=1,inplace=True)
df

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,DEFAULT
0,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
1,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,220000,1,3,1,39,0,0,0,0,0,...,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29996,150000,1,3,2,43,-1,-1,-1,-1,0,...,8979,5190,0,1837,3526,8998,129,0,0,0
29997,30000,1,2,2,37,4,3,2,-1,0,...,20878,20582,19357,0,0,22000,4200,2000,3100,1
29998,80000,1,3,1,41,1,-1,0,0,0,...,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


In [3]:
df.columns

Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'DEFAULT'],
      dtype='object')

In [4]:
features = ['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']

In [5]:
X = df.loc[:, features]
y = df.loc[:, ['DEFAULT']]

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
train_test_split(X, y, test_size=0.10,stratify=y,random_state=1)


In [7]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

In [8]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(criterion='gini', 
                              max_depth=4, 
                              random_state=1)
tree.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=4, random_state=1)

Part 1: Random test train splits

In [9]:
# Train a decision tree model on the training set
from sklearn.metrics import accuracy_score


# Evaluate in-and-out-of-sample accuracy on the test set

in_sample_acc = accuracy_score(y_train, tree.predict(X_train))
out_of_sample_acc = accuracy_score(y_test, tree.predict(X_test))

In [10]:
print(in_sample_acc,out_of_sample_acc)

0.8225555555555556 0.8283333333333334


In [11]:

# Run the loop to calculate accuracy for 10 different samples
random_states = range(1, 11)
accuracy_scores = []
for state in random_states:
    X_train, X_test, y_train, y_test = train_test_split(df.drop("DEFAULT", axis=1), df["DEFAULT"], test_size=0.1, random_state=state)
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)
    accuracy_scores.append(accuracy_score(y_test, model.predict(X_test)))

# Display the individual accuracy scores
print("Individual Accuracy Scores:", accuracy_scores)


Individual Accuracy Scores: [0.724, 0.732, 0.7216666666666667, 0.7153333333333334, 0.7253333333333334, 0.7183333333333334, 0.725, 0.724, 0.735, 0.7316666666666667]


In [12]:
# Display the mean and standard deviation
print('CV accuracy: %.3f +/- %.3f' % (np.mean(accuracy_scores), np.std(accuracy_scores)))

CV accuracy: 0.725 +/- 0.006


Part 2: Cross validation

In [13]:
#using cross_val_scores with k-fold CV (k=10)
import pandas as pd
from sklearn.model_selection import cross_val_score, StratifiedKFold


# Split the data into features and target
X = df.drop("DEFAULT", axis=1)
y = df["DEFAULT"]

# Train decision tree model using StratifiedKFold cross-validation with k=10
model = DecisionTreeClassifier()
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
scores = cross_val_score(model, X, y, cv=cv)

# Display the individual accuracy scores for each fold
print("Individual Accuracy Scores:", scores)

Individual Accuracy Scores: [0.718      0.71966667 0.73       0.72566667 0.72933333 0.721
 0.72266667 0.73733333 0.72666667 0.736     ]


In [14]:
mean_acc = scores.mean()
std_acc = scores.std()

# Display the mean and standard deviation
print("Mean Accuracy:", mean_acc)
print("Standard Deviation of Accuracy:", std_acc)

Mean Accuracy: 0.7266333333333332
Standard Deviation of Accuracy: 0.006263563061247336


In [15]:
#another way to calculate
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

CV accuracy: 0.727 +/- 0.006


In [17]:
 #out-of-sample accuracy score
out_of_sample_acc = accuracy_score(y_test, tree.predict(X_test))
print(out_of_sample_acc)

0.833


Conclusions:Which method of measuring accuracy provides the 
best estimate of how a model will do against unseen data?

The k-fold model does better against unseen data as it has higher mean accuracy and out-of-sample data.


In [18]:
print("My name is {Gabriel Regalado}")
print("My NetID is: {gregal2}")
print("I hereby certify that I have read the University policy on Academic Integrity and that I am not in violation.")

My name is {Gabriel Regalado}
My NetID is: {gregal2}
I hereby certify that I have read the University policy on Academic Integrity and that I am not in violation.
